In [ ]:
import datetime
import subprocess
import webbrowser
import asyncio
import os
import requests
from bs4 import BeautifulSoup
from dotenv import dotenv_values
from groq import Groq
from googlesearch import search as gsearch
from json import load, dump

# Load .env variables
env_vars = dotenv_values(".env")
GroqAPIKey = env_vars.get("GroqAPIKey")
Username = env_vars.get("Username", "Traveler")
Assistantname = env_vars.get("Assistantname", "TravelMate")

client = Groq(api_key=GroqAPIKey)

# Chat history
chatlog_path = "Data/ChatLog.json"
os.makedirs("Data", exist_ok=True)
if not os.path.exists(chatlog_path):
    with open(chatlog_path, "w") as f:
        dump([], f)

SystemChatBot = [
    {"role": "system", "content": f"""You are {Assistantname}, a professional travel assistant helping users plan trips, check destinations, weather, flights, and itineraries. Respond clearly, with proper grammar, punctuation, and professionalism."""}
]

def RealTimeInfo():
    now = datetime.datetime.now()
    return f"Current Time: {now.strftime('%A, %B %d, %Y - %H:%M:%S')}"

def ChatBot(query):
    try:
        with open(chatlog_path, "r") as f:
            messages = load(f)

        messages.append({"role": "user", "content": query})

        completion = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=SystemChatBot + [{"role": "system", "content": RealTimeInfo()}] + messages,
            max_tokens=1200,
            temperature=0.7,
            stream=True
        )

        response = ""
        for chunk in completion:
            if chunk.choices[0].delta.content:
                response += chunk.choices[0].delta.content

        messages.append({"role": "assistant", "content": response})
        with open(chatlog_path, "w") as f:
            dump(messages, f, indent=4)

        return response.strip()
    except Exception as e:
        return f"Error occurred: {e}"

def GoogleSearch(query):
    results = list(gsearch(query, advanced=True, num_results=3))
    return "\n\n".join([f"📌 {r.title}\n{r.description}" for r in results])

def GetWeather(city):
    try:
        url = f"https://wttr.in/{city}?format=3"
        res = requests.get(url)
        return res.text if res.status_code == 200 else "Could not fetch weather."
    except:
        return "Weather request failed."

def GenerateItinerary(destination):
    query = f"Create a 3-day travel itinerary for {destination} including places to visit, local food, and tips."
    return ChatBot(query)

def SearchHotels(destination):
    query = f"Hotels in {destination}"
    webbrowser.open(f"https://www.google.com/search?q={query}")
    return f"Searching hotels in {destination}..."

def SearchFlights(destination):
    webbrowser.open(f"https://www.google.com/flights?q=flights+to+{destination}")
    return f"Opening Google Flights for {destination}..."

async def ExecuteCommand(commands):
    funcs = []

    for cmd in commands:
        cmd = cmd.lower()
        if cmd.startswith("weather "):
            city = cmd.removeprefix("weather ")
            funcs.append(asyncio.to_thread(GetWeather, city))
        elif cmd.startswith("itinerary "):
            dest = cmd.removeprefix("itinerary ")
            funcs.append(asyncio.to_thread(GenerateItinerary, dest))
        elif cmd.startswith("hotels "):
            dest = cmd.removeprefix("hotels ")
            funcs.append(asyncio.to_thread(SearchHotels, dest))
        elif cmd.startswith("flights "):
            dest = cmd.removeprefix("flights ")
            funcs.append(asyncio.to_thread(SearchFlights, dest))
        elif cmd.startswith("search "):
            topic = cmd.removeprefix("search ")
            funcs.append(asyncio.to_thread(GoogleSearch, topic))
        else:
            funcs.append(asyncio.to_thread(ChatBot, cmd))

    results = await asyncio.gather(*funcs)
    for result in results:
        if result:
            print(result)

if __name__ == "__main__":
    print(f"{Assistantname} Ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ("exit", "quit"):
            break
        asyncio.run(ExecuteCommand([user_input]))
